In [14]:
%pip install boto3 sagemaker
%pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [15]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [16]:
import boto3
import pandas as pd
from io import StringIO
import os
# Initialize S3 client
s3_client = boto3.client('s3')

# Define your S3 bucket and file
bucket_name = 'hackathon-hcm'
file_key = 'data-train/data_sample.csv'

# Load the data from S3
response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

if status == 200:
    print(f"Successful S3 get_object response. Status - {status}")
    data = response['Body'].read().decode('utf-8')
    df = pd.read_csv(StringIO(data))
    print(df)
else:
    print(f"Unsuccessful S3 get_object response. Status - {status}")

Successful S3 get_object response. Status - 200
     node_id                 time         ph         tds
0          1  2024-06-01 00:00:00   7.049671  285.846293
1          1  2024-06-01 00:01:00   6.986174  295.793547
2          1  2024-06-01 00:02:00   7.064769  296.572855
3          1  2024-06-01 00:03:00   7.152303  291.977227
4          1  2024-06-01 00:04:00   6.976585  298.387143
..       ...                  ...        ...         ...
995       10  2024-06-01 01:35:00   9.962999  510.701502
996       10  2024-06-01 01:36:00   9.974120  499.734787
997       10  2024-06-01 01:37:00  10.159865  491.181253
998       10  2024-06-01 01:38:00  10.056092  498.369330
999       10  2024-06-01 01:39:00   9.970452  492.550974

[1000 rows x 4 columns]


In [17]:
import sagemaker
from sagemaker.sklearn.estimator import SKLearn

# Define the role with the necessary permissions
role = 'arn:aws:iam::282679431619:role/hcm-hackathon-sagemaker'

# Create the SKLearn estimator
sklearn_estimator = SKLearn(
    entry_point='train.py',
    role=role,
    instance_type='ml.m5.large',
    framework_version='0.23-1',
    py_version='py3',
    hyperparameters={
        'n_estimators': 200,
        'max_samples': 'auto',
        'contamination': 0.1,
        'max_features': 1.0
    }
)
sklearn_estimator.fit({'training': f's3://{bucket_name}/{file_key}'})

INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2024-06-24-04-53-06-042


2024-06-24 04:53:08 Starting - Starting the training job...
2024-06-24 04:53:24 Starting - Preparing the instances for training...
2024-06-24 04:53:47 Downloading - Downloading input data...
2024-06-24 04:54:12 Downloading - Downloading the training image...
2024-06-24 04:55:04 Training - Training image download completed. Training in progress.
2024-06-24 04:55:04 Uploading - Uploading generated training model2024-06-24 04:54:57,990 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-06-24 04:54:57,993 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-06-24 04:54:58,034 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-06-24 04:54:58,195 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-06-24 04:54:58,206 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-06-24 04:54:58,218 sagemaker-training-toolkit INFO

In [ ]:
predictor = sklearn_estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
 )
print(df[['ph', 'tds']].values)
response = predictor.predict(df[['ph', 'tds']].values)
print(response)

In [ ]:
predictor.delete_endpoint()